<a href="https://colab.research.google.com/github/WinetraubLab/OCT2Hist-ModelInference/blob/main/run_oct2hist_over_folder.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>
<a href="https://github.com/WinetraubLab/OCT2Hist-ModelInference/blob/main/run_oct2hist_over_folder.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
This script loads a bunch of OCT images from folder, generates a virtual H&E version of those files and save back to output folder.

In [1]:
# @title Notebook Inputs { display-mode: "form" }
# @markdown Paths

# Path to the OCT image
oct_input_image_folder = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2020-11-10 10x OCT2Hist Model (Paper V2)/Results/TrainSet/" # @param {type:"string"}
virtal_histology_output_folder = "/content/output/" # @param {type:"string"}

# OCT image's pixel size
microns_per_pixel_z = 1
microns_per_pixel_x = 1

# @markdown Cropping Parameters
import numpy as np
x0 = 0 # @param {type:"slider", min:0, max:1000, step:10}
z0 = 0 # @param {type:"slider", min:0, max:1000, step:10}

# @markdown Low signal threshold, negative means auto select
min_signal_threshold = 0.15 # @param {type:"slider", min:-0.1, max:0.4, step:0.01}
if min_signal_threshold < 0:
  min_signal_threshold = np.nan


In [3]:
# @title Setup Environment
if not 'setup_env_oct2Hist_usemodel' in globals():
  setup_env_oct2Hist_usemodel = True
  !git clone --recurse-submodules https://github.com/WinetraubLab/OCT2Hist-ModelInference
  %cd OCT2Hist-ModelInference

import cv2
import matplotlib.pyplot as plt
import numpy as np
import oct2hist
from utils.show_images import *
from google.colab import drive
from utils.crop import crop
import os
drive.mount('/content/drive/')

# Set up the network environment
oct2hist.setup_network()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Mounting drive: if it takes >30 sec, re-run.
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Drive mounted.


In [ ]:
# @title Run

# List images
def list_images_in_folder(folder_path):
  image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')
  images = []

  for root, dirs, files in os.walk(folder_path):
    for file in files:
      if file.lower().endswith(image_extensions):
        file_path = os.path.join(root, file)
        images.append((file, file_path))

  return images
image_path_and_names = list_images_in_folder(oct_input_image_folder)

# Create output folder
if not os.path.exists(virtal_histology_output_folder):
  os.makedirs(virtal_histology_output_folder)

# Run inference and save results
for image_name, image_path in image_path_and_names:

  # Load image, crop
  oct_image = cv2.imread(image_path)
  cropped = crop(oct_image, target_width=1024, target_height=512, x0=x0, z0=z0)

  # Run Inference
  virtual_histology_image, masked_image , o2h_input = oct2hist.run_network(
      cropped,
      microns_per_pixel_x = microns_per_pixel_x,
      microns_per_pixel_z = microns_per_pixel_z,
      min_signal_threshold = min_signal_threshold,
      )

  # Save output file
  virtual_histology_image_path = os.path.join(virtal_histology_output_folder, image_name)
  cv2.imwrite(virtual_histology_image_path, cv2.cvtColor(virtual_histology_image, cv2.COLOR_RGB2BGR))

  # Show the virtual histology image result
  #showTwoImgs(oct_image, virtual_histology_image)

In [9]:
# @title Save folder as a zip file
import zipfile

def zip_images_in_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Call the function to zip images
zip_images_in_folder(virtal_histology_output_folder, '/content/output.zip')